![Logo](../logos/banner_colibri.png)

---
# 🏝️ Welcome to **colibri**'s playground!

Here, you will be able to test all the features of the package. Adapt parameters and run the cells corresponding to the function you want to test to see the result. Make sure you activate the conda environment `colibri` and the Jupyter kernel before starting. A quick description of the functionalities is provided. More technical information is available in docstrings of each function.

---
<br/>
<br/>

In [1]:
# Import colibri

import sys

sys.path.append("..")
import src


<br/>

**1. Run the entire pipeline** <br/>
xxx

In [2]:
query = "ts = (('meta*analysis' or 'systematic review') and ('soil organic carbon' or 'SOC' or 'soil organic matter' or 'SOM' or 'soil carbon'))"
src.wrapper.run_pipeline(query, ["WoS"])    

🐦 Welcome to the pipeline of colibri!

Phase I - Scrapping
You selected platform(s) Web of Science Core Collection to get the publications from.

Scrapping started at 2023-8-8_17-44-52 UTC.
Connected to Web of Science.


NoSuchElementException: Message: Unable to locate element: [id="onetrust-reject-all-handler"]; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:505:5
element.find/</<@chrome://remote/content/marionette/element.sys.mjs:135:16


<br/>

**2. Web of Science scrapping** <br/>
Scrape publications on Web of Science (Core Collection) with a specific search query. DOIs, titles, abstracts and keywords (when available) of each publication will be stored in a Pickle file `colibri/data/wos/<yyyy>-<mm>-<dd>_<hh>-<mm>-<ss>` with the corresponding timestamp of your scrapping.

In [ ]:
# query = "ts = (('meta*analysis' or 'systematic review') and ('soil organic carbon' or 'SOC' or 'soil organic matter' or 'SOM' or 'soil carbon'))"
# src.scrapper.scrapper(query, ["WoS"])